In diesem Jupyter Notebook beschäftigen wir uns mit der Lemmatisierung einer Dokumentensammlung. Dieser Schritt gehört zum Preprocessing der Dokumente. Die Grundidee der Lemmatisierung ist das Reduzieren von Wörtern auf ihre Stammform, auch Lemma genannt.

Lemmatisierung und Stemming haben gemeinsam, dass sie vor allem Suffixe (Nachsilben) und teilweise auch Präfixe entfernen. Der wesentliche Unterschied zwischen den beiden Verfahren liegt jedoch darin, dass Stemming-Algorithmen ohne Kontext arbeiten. Das bedeutet, dass die resultierenden Wortformen teilweise keinen Sinn ergeben oder falsche Grundformen darstellen. Lemmatisierungsalgorithmen hingegen berücksichtigen den Kontext jedes Wortes im Satz und schlagen es in einem Wörterbuch nach, um die korrekte Grundform zu finden. Dadurch entstehen keine falschen Grundformen.

Beispiel:

Original: "The foxes are running swiftly."
    Stemming: "The fox are run swift."
    Lemmatization: "The fox be run swiftly."

Original: "She enjoys reading books."
    Stemming: "She enjoy read book."
    Lemmatization: "She enjoy reading book."

Wir beschäftigen uns mit den zwei geläufigsten Lemmatizern: dem WordNet Lemmatizer aus der NLTK Library und dem SpaCy Lemmatizer.



In [ ]:
#Standard Imports für alle Notebooks

!pip3 install tira ir-datasets python-terrier nltk spacy


from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy

# Loading the NLTK-Ressources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# Laden der SpaCy-Ressourcen
!python -m spacy download en_core_web_sm

# Laden des SpaCy-Modells
nlp = spacy.load("en_core_web_sm")

ensure_pyterrier_is_loaded()
tira = Client()

pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

WordNet Lemmatizer

Der WordNet Lemmatizer basiert auf der lexikalischen Datenbank WordNet. Diese besitzt Informationen über die semantischen Relationen zwischen Wörtern. Diese sind in Gruppen bedeutungsähnlicher Wörter eingeteilt und haben z.B. Informationen über Synonyme, Antonyme oder Hyperonyme. Diese Gruppen werden auch Synsets genannt.

Vorgehensweise

1.) Wörterbuchabgleich**:
    Der Lemmatizer prüft ein Wort gegen WordNet, um das Lemma zu finden.

2.) POS-Tag (Part of Speech)**:
    Mit Hilfe des POS-Tags können wir die Wortart eines Wortes im Kontext des Satzes identifizieren und klassifizieren. Wortarten sind z.B. Nomen, Verben, Adverben oder Adjektive. Damit können wir dann die genaue Grundform des Wortes bestimmen.

    Beispiel:
    - Verb: running -> run
    - Nomen: running -> running




        


In [10]:
# Get the describtion of the POS Tags
def get_wordnet_pos_nltk(treebank_tag):
    """Konvertiert POS-Tag in ein Format, das vom WordNet-Lemmatizer unterstützt wird."""
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [11]:
# Lemmatize a text
def lemmatize_text_nltk(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos_nltk(tag)) for token, tag in pos_tags]
    return ' '.join(lemmatized_tokens)

In [12]:
# Definition der Funktion zur Vorverarbeitung von Dokumenten
def preprocess_documents_nltk(documents):
    """Anwendet Lemmatization auf alle Dokumente."""
    for doc in documents:
        doc['text'] = lemmatize_text_nltk(doc['text'])
        yield doc

In [13]:
# Definition der Funktion zum Erstellen eines Index
def create_index_nltk(documents, stopwords):
    indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, meta={'docno': 100, 'text': 20480}, stopwords=stopwords)
    index_ref = indexer.index(preprocess_documents_nltk(documents))
    return pt.IndexFactory.of(index_ref)

In [14]:
# Definition der Funktion zum Lesen einer Textdatei und Konvertieren in ein Array
def read_text_file_to_array(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            # Entfernen von Zeilenumbrüchen und Konvertierung in Strings
            array = [line.strip() for line in lines]
            return array
    except FileNotFoundError:
        print(f"File {file_path} not found.")
        return None

SpaCy Lemmatizer

Der SpaCy Lemmatizer basiert vor allem auf vortrainierten Modellen.

Vorgehensweise

1.) POS (Part of Speech):
        SpaCy identifiziert und klassifiziert Wortarten mit Hilfe von vortrainierten Algorithmen und Modellen.

2.) Lemmatization:
        SpaCy verwendet Regeln und Wortlisten, um das Lemma für ein Wort zu finden. Diese Regeln und Wortlisten sind in den Sprachmodellen von SpaCy eingebettet und basieren auf umfangreichen Trainingsdaten.

3.) Kontext:
        SpaCy berücksichtigt den ganzen Satz und nicht nur das isolierte Wort, um die Lemmas zu finden.



In [15]:
# Definition der Funktion zur Lemmatization eines Textes mit SpaCy
def lemmatize_text_spacy(text):
    """Lemmatiziert den gegebenen Text mit SpaCy."""
    doc = nlp(text)
    lemmatized_tokens = [token.lemma_ for token in doc]
    return ' '.join(lemmatized_tokens)

# Definition der Funktion zur Vorverarbeitung von Dokumenten
def preprocess_documents_spacy(documents):
    """Anwendet Lemmatization auf alle Dokumente."""
    for doc in documents:
        doc['text'] = lemmatize_text_spacy(doc['text'])
        yield doc

# Definition der Funktion zum Erstellen eines Index
def create_index_spacy(documents, stopwords):
    indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, meta={'docno': 100, 'text': 20480}, stopwords=stopwords)
    index_ref = indexer.index(preprocess_documents_spacy(documents))
    return pt.IndexFactory.of(index_ref)


     

In [ ]:
# Lesen der Stopwords-Datei und Konvertieren in ein Array
file_path = "../terrier-stopwordlist.txt"
stopwords = read_text_file_to_array(file_path)


costum_index_nltk =create_index_nltk(pt_dataset.get_corpus_iter(), stopwords)
# Erstellen des benutzerdefinierten Index mit Lemmatization
costum_index_spacy = create_index_spacy(pt_dataset.get_corpus_iter(), stopwords)

# Erstellen der BatchRetrieve-Instanzen
XSqrA_M = pt.BatchRetrieve(index, wmodel="XSqrA_M")
XSqrA_M_nltk = pt.BatchRetrieve(costum_index_nltk, wmodel="XSqrA_M")
XSqrA_M_spacy = pt.BatchRetrieve(costum_index_spacy, wmodel="XSqrA_M")

In [ ]:
# Evaluation der Modelle
pt.Experiment(
    [XSqrA_M, XSqrA_M_nltk, XSqrA_M_spacy],
    pt_dataset.get_topics(),
    pt_dataset.get_qrels(),
    ["ndcg_cut.10", "recip_rank", "recall_100", "P_10"],
    names=["XSqrA_M", "XSqrA_M_NLTK", "XSqrA_M_SPACY"]
)